# 1. 위경도 -> 지번 주소 추출 (API 사용)

In [35]:
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address = geocoding_reverse('35.5547651056571, 129.32045439272568')
print(address)

만남의거리, 성남동 젊음의 거리, 중구, 울산, 44532, 대한민국


In [17]:
# test 
import requests
apiurl = "https://api.vworld.kr/req/address?"
params = {
    "service": "address",
    "request": "getaddress",
    "crs": "epsg:4326",
    "point": "127.101313354,37.402352535",
    "format": "json",
    "type": "parcel",
    "key": "57C428EF-A97E-3028-9BF6-734745493260"
}
response = requests.get(apiurl, params=params)
if response.status_code == 200:
    json_data = response.json()
    print(json_data['response']['result'][0]['text']) # 지번주소 

경기도 성남시 분당구 삼평동 624-3


In [33]:
# 픽업지 지번 주소 추출 

import requests
import pandas as pd

apiurl = "https://api.vworld.kr/req/address"
api_key = "57C428EF-A97E-3028-9BF6-734745493260"

data = pd.read_csv('/Users/yj.noh/Desktop/pick_address.csv', encoding="utf-8")

shop_results = []

for index, row in data.iterrows():
    lat = row['shop_loc_pnt_lat']
    lon = row['shop_loc_pnt_lon']
    
    # 픽업지 주소 추출
    params_shop = {
        "service": "address",
        "request": "getaddress",
        "crs": "epsg:4326",
        "point": f"{lon},{lat}",  # 경도, 위도 순서로 입력
        "format": "json",
        "type": "parcel",
        "key": api_key
    }
    
    response_shop = requests.get(apiurl, params=params_shop)
    
    if response_shop.status_code == 200:
        result_shop = response_shop.json()
        shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
            
    shop_results.append({
        "Latitude": lat,
        "Longitude": lon,
        "Address": shop_address
        })
    
shop_result = pd.DataFrame(shop_results)
shop_result.to_csv('shop_result.csv', index=False, encoding="cp949")




In [34]:
# 전달지 지번 주소 추출 

import requests
import pandas as pd

apiurl = "https://api.vworld.kr/req/address"
api_key = "57C428EF-A97E-3028-9BF6-734745493260"

data = pd.read_csv('/Users/yj.noh/Desktop/dlvry_address.csv', encoding="utf-8")

dlvry_results = []

for index, row in data.iterrows():
    lat = row['dlvry_loc_pnt_lat']
    lon = row['dlvry_loc_pnt_lon']
    
    # 픽업지 주소 추출
    params_shop = {
        "service": "address",
        "request": "getaddress",
        "crs": "epsg:4326",
        "point": f"{lon},{lat}",  # 경도, 위도 순서로 입력
        "format": "json",
        "type": "parcel",
        "key": api_key
    }
    
    response_shop = requests.get(apiurl, params=params_shop)
    
    if response_shop.status_code == 200:
        result_shop = response_shop.json()
        shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
            
    dlvry_results.append({
        "Latitude": lat,
        "Longitude": lon,
        "Address": shop_address
        })
    
dlvry_result = pd.DataFrame(dlvry_results)
dlvry_result.to_csv('dlvry_result.csv', index=False, encoding="cp949")

In [ ]:
# 픽업지 - 전달지 지번 주소 추출 

import requests
import pandas as pd

apiurl = "https://api.vworld.kr/req/address"
api_key = "57C428EF-A97E-3028-9BF6-734745493260"

data = pd.read_csv('/Users/yj.noh/Desktop/pick_dlvry_lat_loc.csv', encoding="cp949")

shop_results = []
dlvry_results = []

for index, row in data.iterrows():
    shop_lat = row['shop_loc_pnt_lat']
    shop_lon = row['shop_loc_pnt_lon']
    dlvry_lat = row['dlvry_loc_pnt_lat']
    dlvry_lon = row['dlvry_loc_pnt_lon']
    
    # 픽업지 주소 추출
    params_shop = {
        "service": "address",
        "request": "getaddress",
        "crs": "epsg:4326",
        "point": f"{shop_lon},{shop_lat}",  # 경도, 위도 순서로 입력
        "format": "json",
        "type": "parcel",
        "key": api_key
    }
    
    response_shop = requests.get(apiurl, params=params_shop)
    
    if response_shop.status_code == 200:
        result_shop = response_shop.json()
        shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
    
    # 전달지 주소 추출
    params_dlvry = {
        "service": "address",
        "request": "getaddress",
        "crs": "epsg:4326",
        "point": f"{dlvry_lon},{dlvry_lat}",  # 경도, 위도 순서로 입력
        "format": "json",
        "type": "parcel",
        "key": api_key
    }
    
    response_dlvry = requests.get(apiurl, params=params_dlvry)
    
    if response_dlvry.status_code == 200:
        result_dlvry = response_dlvry.json()
        dlvry_address = result_dlvry['response']['result'][0]['text']  # 배송지의 지번 주소
        
    shop_results.append({
        "Latitude": shop_lat,
        "Longitude": shop_lon,
        "Address": shop_address
        })
    
    dlvry_results.append ({
        "Latitude": dlvry_lat,
        "Longitude": dlvry_lon,
        "Address": dlvry_address
    })

shop_result = pd.DataFrame(shop_results)
dlvry_result = pd.DataFrame(dlvry_results)

result = pd.concat([shop_result, dlvry_result])
result.to_csv('address_result.csv', index=False, encoding="cp949")




# 2. 지번주소 -> 건물 형태 데이터 join 

In [53]:
# 공공데이터 건축물대장정보 데이터 가져오기
from PublicDataReader import BuildingLedger
import PublicDataReader as pdr
import pandas as pd 

# 시군구코드, 법정동코드 조회하기
service_key = "uw7Y8kgQ1Fqg6z9GjnSN8jJ8S%2FuV%2Bl%2B8PnHC4By9xwtYF5ZAExmO2Ip1mJcQC3HDTDcKmIxB9rBBuI0gDaIYVA%3D%3D"
api = BuildingLedger(service_key)

gu_name = ['서초구'] #, '강남구', '관악구', '동작구', '과천시']
code = pdr.code_bdong()

code['말소일자'] = pd.to_datetime(code['말소일자'])
filtered_data = code[code['시군구명'].str.contains('|'.join(gu_name)) & (code['말소일자'].isna()) & (code['읍면동명'].fillna('') != '')]
#filtered_data.to_csv('시군구법정코드추출.csv', index=False, encoding = "cp949")

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023. 7. 3. 시행)


In [ ]:
# 시군구 법정동에 존재하는 건물 데이터 추출
all_data = []

for index, row in filtered_data.iterrows():
    sigungu_code = 11650
    bdong_code = row['법정동코드'][-5:]

    df = api.get_data(
        ledger_type = "총괄표제부",
        sigungu_code = sigungu_code,
        bdong_code = bdong_code,
        bun = "",
        ji = "",
    )
    all_data.append(df)

combined_data = pd.concat(all_data, ignore_index=True)
combined_data.to_csv('address/all_building_info.csv', index=False, encoding="cp949")


In [130]:
df1 = api.get_data(
        ledger_type="총괄표제부", 
        sigungu_code = '41290',
        bdong_code = '10600',
        bun="", 
        ji="",
    )
df1.to_csv('address/건축물대장/과천시/주암동.csv', index=False, encoding = "cp949")

In [132]:
df1 = api.get_data(
        ledger_type="총괄표제부", 
        sigungu_code = '11650',
        bdong_code = '10900',
        bun="", 
        ji="",
    )
df1.to_csv('address/건축물대장/서초구/내곡동.csv', index=False, encoding = "cp949")